In [0]:
import pandas as pd
from pprint import pprint

In [0]:
file = pd.ExcelFile("TC-guide.xlsx")
file = pd.read_excel(file, "Data")

In [0]:
# Fill all the NaNs with 0s.
file = file.fillna(0)

# Adjacency matrix for graph.
mat = {}

keyfloor = set()
import collections
switchfloor = collections.defaultdict(list)

# For every row in our excel file.
for idx,x in enumerate(file.as_matrix()):
  for idy,y in enumerate(x):

    # If this is a key.
    if y!=0 and "Key" in y and "+" not in y:
      if str(idx) + file.columns[idy] not in mat:
        mat[str(idx)+file.columns[idy]] = {}
      mat[str(idx)+file.columns[idy]][str(idx+1)+file.columns[idy]] = -1
      keyfloor.add(str(idx)+file.columns[idy])
    
    # If the cell is not only a key.
    elif y != 0 and y != "Exit" and y!= "Mew":
      ex = False
      #For all cells with format "something+switch"
      if "+" in y:          
          # For cells with format "Exit+switch"
          if "Exit" in y:
            if str(idx) + file.columns[idy] not in mat:
              mat[str(idx)+file.columns[idy]] = {}
            mat[str(idx)+file.columns[idy]][str(idx+1)+file.columns[idy]] = 10000
            ex = True
          
          # For cells with format "Key+switch"
          if "Key" in y:
            if str(idx) + file.columns[idy] not in mat:
              mat[str(idx)+file.columns[idy]] = {}
            mat[str(idx)+file.columns[idy]][str(idx+1)+file.columns[idy]] = -1
            keyfloor.add(str(idx)+file.columns[idy])
          y = y[y.index("+")+1:]
          if "Mew" in y:
            if "+" in y:
              y = y[y.index("+")+1:]
            else:
              y = ""
      # Converting all switches to edges in graph
      for c in y:
        # Convert to lower for consistency.
        c = c.lower()
        
        # If the character is a letter from c to q, based on columns in the xl
        if ord(c) >= 97 and ord(c) <= ord('q') :
          # If our floor+chamber string is not in the graph, add the entry.
          if str(idx) + file.columns[idy] not in mat:
            mat[str(idx)+file.columns[idy]] = {}
          # Form edge from switch on this floor to next floor in resp. chamber.
          # Weighted 1 to add requirement of needing a key.
          mat[str(idx)+file.columns[idy]][str(idx+1)+file.columns[ord(c)-97]]= 1
          switchfloor[str(idx)].append(file.columns[ord(c)-97])
      # Edge for this floor to next floor without switch.
      if not ex:
        if str(idx) + file.columns[idy] not in mat:
            mat[str(idx)+file.columns[idy]] = {}
        mat[str(idx)+file.columns[idy]][str(idx+1)+file.columns[idy]] = 0

    # If it's an exit switch cell.
    elif y == 'Exit':
      if str(idx) + file.columns[idy] not in mat:
            mat[str(idx)+file.columns[idy]] = {}
      
      # Heavily penalize so that no more steps can be taken here on.
      mat[str(idx)+file.columns[idy]][str(idx+1)+file.columns[idy]] = 10000

    # Normal walk to next floor, no penalty.
    elif y == 0 or y == "Mew":
      if str(idx) + file.columns[idy] not in mat:
            mat[str(idx)+file.columns[idy]] = {}
      mat[str(idx)+file.columns[idy]][str(idx+1)+file.columns[idy]] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [0]:
start = '0Normal'
distance = {}
pred = {}
keys = {}

# Initialize Bellman Ford distances.
for x in range(99):
  for y in file.columns:
    distance[str(x)+y] = 0 if str(x)+y == start else float('inf')
    keys[str(x)+y] = 0

# Bellman Ford on graph we made above.
for i in range(len(mat)):
  for v in mat:
    for n in mat[v]:
      if "-" in n:
        continue
      distance[n] = min(distance[n], distance[v] + mat[v][n])
      distance[n] = abs(distance[n])
      if distance[n] >= distance[v]+mat[v][n]:
        pred[n] = v 

In [0]:
# Retrieve path from our algorithm above.
print("Enter desired destination in format: (Number-2)Type)")
print("E.g: 97Fire for Fire exit on the 99th floor.")
it = input()
numk = 0
nums = 0
path = []
while it != start:
  path.append(it)
  it = pred[it]
path.append(it)
print("Distance: ", distance[path[1]]/99)
print("Steps to follow:")
prev = None
prevcol = None
for idx,x in enumerate(reversed(path)):
  if x in keyfloor:
      print(str(idx)+".",x, " (Key)")
      numk+=1
  for y in file.columns:
    if y in x:
      mycol = y
      break
  if prevcol != None and prevcol != mycol: 
    print(str(idx)+".",prev, " (Switch)")
    nums+=1
  if idx == len(path)-1:
    print(str(idx)+".", x, "(Exit)", "\nNeeded: % 2d keys and % 2d switches" % (numk,nums))
  prevcol = mycol
  prev = x

Enter desired destination in format: (Number-2)Type)
E.g: 97Fire for Fire exit on the 99th floor.
12Normal
Distance:  0.0
Steps to follow:
0. 0Normal  (Key)
2. 2Normal  (Key)
12. 12Normal (Exit) 
Needed:  2 keys and  0 switches


In [0]:
# <--------- IGNORE BELOW LINE ---------->

import pandas as pd
from pprint import pprint
backfile = pd.ExcelFile("TC-guide.xlsx")
backfile = pd.read_excel(backfile, "SwitchForm")

In [0]:
# Fill all the NaNs with 0s.
backfile = backfile.fillna(0)

# Adjacency matrix for graph.
mat = {}
keys = set([])
ex = set([])
switches = {}

# For every row in our excel file.
for idx,x in enumerate(backfile.as_matrix()):
  for idy,y in enumerate(x):

    # If this is a key.
    if y != 0:
      if y == "Key":
            if str(idx) + backfile.columns[idy] not in mat:
              mat[str(idx)+backfile.columns[idy]] = {}
            mat[str(idx)+backfile.columns[idy]][str(idx-1)+backfile.columns[idy]] = -1
            keys.add(str(idx)+backfile.columns[idy])
            continue
      #For all cells with format "something+switch"
      if "+" in y:          
          # For cells with format "Exit+switch"
          if "Exit" in y:
            print(y)
            print(y)
            ex.add((idx,backfile.columns[idy]))
          # For cells with format "Key+switch"
          if "Key" in y:
            keys.add(str(idx)+backfile.columns[idy])
          y = y[y.index("+")+1:]

      # Converting all switches to edges in graph
      for c in y:
        # Convert to lower for consistency.
        c = c.lower()
        
        # If the character is a letter from c to q, based on columns in the xl
        if ord(c) >= 97 and ord(c) <= ord('q') :
          # If our floor+chamber string is not in the graph, add the entry.
          if str(idx) + backfile.columns[idy] not in mat:
            mat[str(idx)+backfile.columns[idy]] = {}
          # Form edge from switch on this floor to next floor in resp. chamber.
          # Weighted 1 to add requirement of needing a key.
          mat[str(idx)+backfile.columns[idy]][str(idx-1)+backfile.columns[ord(c)-97]]= 1
      
      # Edge for this floor to next floor without switch.
      mat[str(idx)+backfile.columns[idy]][str(idx-1)+backfile.columns[idy]] = 0

    # Normal walk to next floor, no penalty.
    elif y == 0:
      if str(idx) + backfile.columns[idy] not in mat:
            mat[str(idx)+backfile.columns[idy]] = {}
      mat[str(idx)+backfile.columns[idy]][str(idx-1)+backfile.columns[idy]] = 0
for exits in ex:
  print(str(exits[0])+exits[1])
  mat[str(exits[0]+1)+exits[1]].pop(str(exits[0])+exits[1])
print(mat)

In [0]:
start = "26Fire"
dest = "0Normal"
queue = [[start]]
pred = {}
pred[start] = (None,0)
visited = set([])
exit = False

while queue[-1]:
  queue.append([])
  while queue[-2]:
    node = queue[-2].pop()
    for neighbor in mat[node]:
      if "-" in neighbor:
        continue
      if neighbor != dest and neighbor not in visited:
        if mat[node][neighbor] == -1:
          pred[neighbor] = (node,pred[node][1]+1)
        if neighbor in keys:
          pred[neighbor] = (node,pred[node][1]-1)
        elif mat[node][neighbor] == 'Exit':
          continue
        else:
          pred[neighbor] = (node,pred[node][1])
        queue[-1].append(neighbor)
        visited.add(neighbor)
      elif neighbor == dest:
        print(neighbor,pred[node])
        if pred[node][1]-1 <= 0:
          pred[neighbor] = (node,pred[node][1])
          exit = True
          break
    if exit:
      break
  if exit:
    break
  
print(pred)

In [0]:
# Retrieve path from our algorithm above.
it = '0Normal'
path = []
while it != start:
  path.append(it)
  it = pred[it][0]
path.append(it)
print("Steps to follow:")
prev = None
prevcol = None
for idx,x in enumerate(path):
  if x in keys:
      print(str(idx)+".",x, " (Key)")
  for y in backfile.columns:
    if y in x:
      mycol = y
      break
  if prevcol != None and prevcol != mycol: 
    print(str(idx)+".",prev)
  if idx == len(path)-1:
    print(str(idx)+".", x)
  prevcol = mycol
  prev = x

In [0]:
print('6Fire' in keys)